# read txt

In [1]:
# Import libs
import findspark
findspark.init("/home/antonis/spark-2.3.0-bin-hadoop2.7")
import sys
from pyspark import SparkContext
from pyspark.sql import SQLContext
from pyspark.sql.types import *


In [2]:
# Load relevant objects
sc = SparkContext('local').getOrCreate()

sqlContext = SQLContext(sc)

In [23]:
log_txt = sc.textFile("/home/george/snp/snpCode.txt")
sc.setLogLevel("WARN")

In [24]:
header = log_txt.first()

In [25]:
header

'field0\tfield1\tfield2\tfield3\tfield4'

In [26]:
log_txt = log_txt.filter(lambda line: line != header)
log_txt.take(10)



['2263887\trs6694740\t0\tC\tC',
 '2263887\trs10863861\t1\tT\tC',
 '2263887\trs535295013\t0\tT\tT',
 '2263887\trs531640184\t0\tG\tG',
 '2263887\trs11118208\t0\tA\tA',
 '2263887\trs7529836\t1\tC\tT',
 '2263887\trs187391176\t0\tC\tC',
 '2263887\trs572351358\t0\tG\tG',
 '2263887\trs149530621\t0\tG\tG',
 '2263887\trs116137046\t0\tC\tC']

In [27]:
temp_var = log_txt.map(lambda k: k.split("\t"))
log_df=temp_var.toDF(header.split("\t"))
log_df.show()

+-------+-----------+------+------+------+
| field0|     field1|field2|field3|field4|
+-------+-----------+------+------+------+
|2263887|  rs6694740|     0|     C|     C|
|2263887| rs10863861|     1|     T|     C|
|2263887|rs535295013|     0|     T|     T|
|2263887|rs531640184|     0|     G|     G|
|2263887| rs11118208|     0|     A|     A|
|2263887|  rs7529836|     1|     C|     T|
|2263887|rs187391176|     0|     C|     C|
|2263887|rs572351358|     0|     G|     G|
|2263887|rs149530621|     0|     G|     G|
|2263887|rs116137046|     0|     C|     C|
|2263887|rs555573899|     0|     G|     G|
|2263887|rs115525961|     0|     C|     C|
|2263887|rs574151572|     0|     C|     C|
|2263887|rs140320512|     0|     A|     A|
|2263887| rs79439279|     0|     C|     C|
|2263887|rs539625922|     0|     G|     G|
|2263887| rs75606730|     0|     A|     A|
|2263887|rs537187991|     0|     T|     T|
|2263887|rs370849882|     0|   ATT|   ATT|
|2263887|rs192396930|     0|     G|     G|
+-------+--

In [30]:
log_df.printSchema()

root
 |-- field0: string (nullable = true)
 |-- field1: string (nullable = true)
 |-- field2: string (nullable = true)
 |-- field3: string (nullable = true)
 |-- field4: string (nullable = true)



In [35]:
log_df = log_df.withColumn("disease", log_df["field2"].cast(IntegerType()))

In [37]:
log_df.printSchema()

root
 |-- field0: string (nullable = true)
 |-- field1: string (nullable = true)
 |-- field2: string (nullable = true)
 |-- field3: string (nullable = true)
 |-- field4: string (nullable = true)
 |-- disease: integer (nullable = true)
 |-- field1Int: integer (nullable = true)



In [38]:
data = log_df.select(["field0","field1","disease"])

In [39]:
data.printSchema()

root
 |-- field0: string (nullable = true)
 |-- field1: string (nullable = true)
 |-- disease: integer (nullable = true)



In [40]:
data.describe()

DataFrame[summary: string, field0: string, field1: string, disease: string]

In [41]:
data.show()

+-------+-----------+-------+
| field0|     field1|disease|
+-------+-----------+-------+
|2263887|  rs6694740|      0|
|2263887| rs10863861|      1|
|2263887|rs535295013|      0|
|2263887|rs531640184|      0|
|2263887| rs11118208|      0|
|2263887|  rs7529836|      1|
|2263887|rs187391176|      0|
|2263887|rs572351358|      0|
|2263887|rs149530621|      0|
|2263887|rs116137046|      0|
|2263887|rs555573899|      0|
|2263887|rs115525961|      0|
|2263887|rs574151572|      0|
|2263887|rs140320512|      0|
|2263887| rs79439279|      0|
|2263887|rs539625922|      0|
|2263887| rs75606730|      0|
|2263887|rs537187991|      0|
|2263887|rs370849882|      0|
|2263887|rs192396930|      0|
+-------+-----------+-------+
only showing top 20 rows



In [42]:
from pyspark.ml.feature import (VectorAssembler,VectorIndexer,
                               OneHotEncoder , StringIndexer)

In [43]:
rs_indexer = StringIndexer(inputCol='field1',outputCol='rsIndex')
rs_encoder = OneHotEncoder(inputCol='rsIndex',outputCol='rsVec')

In [1]:
assempler = VectorAssembler(inputCols=['field0','disease','rsVec'],outputCol='features')


NameError: name 'VectorAssembler' is not defined

In [46]:
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression


In [47]:
pipeline = Pipeline(stages=[rs_indexer,rs_encoder,
                           
                           assempler])



In [48]:
data_i = rs_indexer.fit(data)

In [50]:
data = data_i.transform(data)

In [52]:
data = rs_encoder.transform(data)

In [53]:
data.show()

+-------+-----------+-------+-------+-------------------+
| field0|     field1|disease|rsIndex|              rsVec|
+-------+-----------+-------+-------+-------------------+
|2263887|  rs6694740|      0|  496.0| (5414,[496],[1.0])|
|2263887| rs10863861|      1| 3491.0|(5414,[3491],[1.0])|
|2263887|rs535295013|      0| 2984.0|(5414,[2984],[1.0])|
|2263887|rs531640184|      0| 3478.0|(5414,[3478],[1.0])|
|2263887| rs11118208|      0| 4078.0|(5414,[4078],[1.0])|
|2263887|  rs7529836|      1|  830.0| (5414,[830],[1.0])|
|2263887|rs187391176|      0| 4763.0|(5414,[4763],[1.0])|
|2263887|rs572351358|      0| 3153.0|(5414,[3153],[1.0])|
|2263887|rs149530621|      0| 4746.0|(5414,[4746],[1.0])|
|2263887|rs116137046|      0|  984.0| (5414,[984],[1.0])|
|2263887|rs555573899|      0| 4845.0|(5414,[4845],[1.0])|
|2263887|rs115525961|      0|  468.0| (5414,[468],[1.0])|
|2263887|rs574151572|      0| 1871.0|(5414,[1871],[1.0])|
|2263887|rs140320512|      0| 4791.0|(5414,[4791],[1.0])|
|2263887| rs79

# DOkimes

In [2]:
# Import libs
import findspark
findspark.init("/home/george/spark-2.2.1-bin-hadoop2.7")
import sys
from pyspark import SparkContext
from pyspark.streaming import StreamingContext


sc = SparkContext(appName="StreamingErrorCount").getOrCreate();
        # 2 is the batch interval : 2 seconds
ssc = StreamingContext(sc, 2)


In [4]:

        # Checkpoint for backups
        #ssc.checkpoint("file:///tmp/spark")

        # Define the socket where the system will listen
        # Lines is not a rdd but a sequence of rdd, not static, constantly changing
lines = ssc.textFileStream('/home/george/snp/')
        #lines.collect


In [ ]:

        # Counting errors
        llist = lines.collect()

        # print the list
        for line in llist:
            print(line)

        ssc.start()
        ssc.awaitTermination()

In [2]:
ssc.stop()

In [7]:
# Begin
if __name__ == "__main__":
        sc = SparkContext(appName="StreamingErrorCount").getOrCreate();
        # 2 is the batch interval : 2 seconds
        ssc = StreamingContext(sc, 2)

        # Checkpoint for backups
        ssc.checkpoint("file:///tmp/spark")

        # Define the socket where the system will listen
        # Lines is not a rdd but a sequence of rdd, not static, constantly changing
        lines = ssc.socketTextStream("localhost", 9999)



        # Update function
        def countWords(newValues, lastSum):
            if lastSum is None :
                lastSum = 0
            return sum(newValues, lastSum)

        word_counts = lines.flatMap(lambda line: line.split(" "))\
                    .map(lambda word : (word, 1))\
                    .updateStateByKey(countWords)

        word_counts.pprint()
        ssc.start()
        ssc.awaitTermination()

-------------------------------------------
Time: 2018-03-18 22:46:40
-------------------------------------------

-------------------------------------------
Time: 2018-03-18 22:46:42
-------------------------------------------

-------------------------------------------
Time: 2018-03-18 22:46:44
-------------------------------------------

-------------------------------------------
Time: 2018-03-18 22:46:46
-------------------------------------------

-------------------------------------------
Time: 2018-03-18 22:46:48
-------------------------------------------
('ss', 1)

-------------------------------------------
Time: 2018-03-18 22:46:50
-------------------------------------------
('s', 1)
('', 1)
('ss', 1)

-------------------------------------------
Time: 2018-03-18 22:46:52
-------------------------------------------
('s', 1)
('', 1)
('ss', 1)

-------------------------------------------
Time: 2018-03-18 22:46:54
-------------------------------------------
('s', 1)
('', 1)


-------------------------------------------
Time: 2018-03-18 22:48:38
-------------------------------------------
('s', 1)
('', 1)
('ss', 1)

-------------------------------------------
Time: 2018-03-18 22:48:40
-------------------------------------------
('s', 1)
('', 1)
('ss', 1)

-------------------------------------------
Time: 2018-03-18 22:48:42
-------------------------------------------
('s', 1)
('', 1)
('ss', 1)

-------------------------------------------
Time: 2018-03-18 22:48:44
-------------------------------------------
('s', 1)
('', 1)
('ss', 1)

-------------------------------------------
Time: 2018-03-18 22:48:46
-------------------------------------------
('s', 1)
('', 1)
('ss', 1)

-------------------------------------------
Time: 2018-03-18 22:48:48
-------------------------------------------
('s', 1)
('', 1)
('ss', 1)

-------------------------------------------
Time: 2018-03-18 22:48:50
-------------------------------------------
('s', 1)
('', 1)
('ss', 1)

------

-------------------------------------------
Time: 2018-03-18 22:50:34
-------------------------------------------
('s', 1)
('', 1)
('ss', 1)

-------------------------------------------
Time: 2018-03-18 22:50:36
-------------------------------------------
('s', 1)
('', 1)
('ss', 1)

-------------------------------------------
Time: 2018-03-18 22:50:38
-------------------------------------------
('s', 1)
('', 1)
('ss', 1)

-------------------------------------------
Time: 2018-03-18 22:50:40
-------------------------------------------
('s', 1)
('', 1)
('ss', 1)

-------------------------------------------
Time: 2018-03-18 22:50:42
-------------------------------------------
('s', 1)
('', 1)
('ss', 1)

-------------------------------------------
Time: 2018-03-18 22:50:44
-------------------------------------------
('s', 1)
('', 1)
('ss', 1)

-------------------------------------------
Time: 2018-03-18 22:50:46
-------------------------------------------
('s', 1)
('', 1)
('ss', 1)

------

-------------------------------------------
Time: 2018-03-18 22:52:30
-------------------------------------------
('s', 1)
('', 1)
('ss', 1)

-------------------------------------------
Time: 2018-03-18 22:52:32
-------------------------------------------
('s', 1)
('', 1)
('ss', 1)

-------------------------------------------
Time: 2018-03-18 22:52:34
-------------------------------------------
('s', 1)
('', 1)
('ss', 1)



KeyboardInterrupt: 

-------------------------------------------
Time: 2018-03-18 22:52:36
-------------------------------------------
('s', 1)
('', 1)
('ss', 1)



In [31]:
import sys
from pyspark.sql import SparkSession, functions, types

spark = SparkSession.builder.appName('streaming line lengths').getOrCreate()
# sc = spark.sparkContext
assert sys.version_info >= (3, 4) # make sure we have Python 3.4+
assert spark.version >= '2.2' # make sure we have Spark 2.2+


In [32]:
from pyspark.sql.functions import explode
from pyspark.sql.functions import split


In [52]:
stream.stop()

ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/home/george/spark-2.2.1-bin-hadoop2.7/python/lib/py4j-0.10.4-src.zip/py4j/java_gateway.py", line 1035, in send_command
    raise Py4JNetworkError("Answer from Java side is empty")
py4j.protocol.Py4JNetworkError: Answer from Java side is empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/george/spark-2.2.1-bin-hadoop2.7/python/lib/py4j-0.10.4-src.zip/py4j/java_gateway.py", line 883, in send_command
    response = connection.send_command(command)
  File "/home/george/spark-2.2.1-bin-hadoop2.7/python/lib/py4j-0.10.4-src.zip/py4j/java_gateway.py", line 1040, in send_command
    "Error while receiving", e, proto.ERROR_ON_RECEIVE)
py4j.protocol.Py4JNetworkError: Error while receiving


Py4JError: An error occurred while calling o975.stop

In [34]:
 
lines = spark.

# Split the lines into words
words = lines.select(
   explode(
       split(lines.value, "\t")
   ).alias("word")
)
wordCounts = words.groupBy("word").count()
stream = wordCounts.writeStream.format('console') \
        .outputMode('update').start()
stream.awaitTermination(3600)


StreamingQueryException: 'Option \'basePath\' must be a directory\n=== Streaming Query ===\nIdentifier: [id = 72268626-3564-4266-8095-e6ecd47dcb2a, runId = ef315ef7-959f-4d86-82d2-bb37c8ff7247]\nCurrent Committed Offsets: {}\nCurrent Available Offsets: {FileStreamSource[file:/home/george/snp/snpTest.txt]: {"logOffset":0}}\n\nCurrent State: ACTIVE\nThread State: RUNNABLE\n\nLogical Plan:\nAggregate [word#75], [word#75, count(1) AS count#81L]\n+- Project [word#75]\n   +- Generate explode(split(value#71, \t)), false, false, [word#75]\n      +- StreamingExecutionRelation FileStreamSource[file:/home/george/snp/snpTest.txt], [value#71]\n'

In [ ]:
lengths = lines.select(
    functions.length(lines['value']).alias('len')
)
counts = lengths.groupBy('len').count()

stream = counts.writeStream.format('console') \
        .outputMode('update').start()
stream.awaitTermination(3600)